In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize
import re
from nltk.data import load
import random
import json
import sys
import random
sys.path.append('../')
tokenizer = load('tokenizers/punkt/{0}.pickle'.format('english'))
tokenizer._params.abbrev_types.add('..')
tokenizer._params.abbrev_types.add('No')
tokenizer._params.abbrev_types.add('no')
tokenizer._params.abbrev_types.add('Dr')
tokenizer._params.abbrev_types.add('dr')
tokenizer._params.abbrev_types.add('op')
tokenizer._params.abbrev_types.add('J.S.')

def default_sentence_split(passage):
    return tokenizer.tokenize(passage)

In [4]:
para_train = pd.read_csv('../../../dataset/paraphrase_extended_train.csv',index_col=None).dropna()
para_test = pd.read_csv('../../../dataset/paraphrase_test.csv').dropna()

In [6]:
import tqdm
from src.config import GenerationTasks

from src.dataset_processor import QuestionGenerationData


def process_paraphrase(datapack,verbose=False):
    datapack_strip = datapack.values
    dataset = []
    for idx, dat in tqdm.tqdm(enumerate(datapack_strip )) if verbose else enumerate(datapack_strip ):
        q1= dat[0]
        q2 = dat[1]

        vani_dob_q = QuestionGenerationData(task=GenerationTasks.question_paraphrase,
                                            input_text= q1, 
                                            output_text=q2,
                                            contextual_text= "")
        dataset.append(vani_dob_q)
    return dataset

In [9]:
train_para = process_paraphrase(para_train,True)
test_para = process_paraphrase(para_test,True)

146161it [00:00, 344875.62it/s]
4220it [00:00, 602921.38it/s]


In [10]:
from dataclass_csv import DataclassReader, dateformat,DataclassWriter

with open("../curated_data/para_train.csv", "w",encoding='utf-8') as f:
    w = DataclassWriter(f, train_para, QuestionGenerationData)
    w.write()

with open("../curated_data/para_dev.csv", "w",encoding='utf-8') as f:
    w = DataclassWriter(f, test_para, QuestionGenerationData)
    w.write()

In [11]:
import glob

In [15]:
files = glob.glob('../curated_data/*.csv')
def load_dataset(data_path: str):
    pack = []
    with open(data_path, encoding="utf-8") as f:
        dataset = DataclassReader(f, QuestionGenerationData)
        for row in dataset:
            pack.append(row)
    return pack

def load_all_data(dataset_path,mode='train'):
    files = glob.glob(dataset_path+f'*{mode}.csv')
    print('processing files: ',files)
    dataset = []
    for file in files:
        dataset+=load_dataset(file)
    random.shuffle(dataset)
    random.shuffle(dataset)
    return dataset

In [25]:
train_dataset = load_all_data('../curated_data/')
test_dataset = load_all_data('../curated_data/',mode='dev')

processing files:  ['../curated_data\\drop_train.csv', '../curated_data\\extra_data_train.csv', '../curated_data\\para_train.csv', '../curated_data\\squad_train.csv']
processing files:  ['../curated_data\\drop_dev.csv', '../curated_data\\para_dev.csv', '../curated_data\\squad_dev.csv']


In [20]:
train_dataset[-1]

QuestionGenerationData(task='<generate_questions>', input_text='In discussions of matter and antimatter, normal matter has been referred to by Alfvén as koinomatter (Gk. common matter). It is fair to say that in physics, there is no broad consensus as to a general definition of matter, and the term "matter" usually is used in conjunction with a specifying modifier.', output_text='What field of study has a variety of unusual contexts?', contextual_text='It is fair to say that in physics, there is no broad consensus as to a general definition of matter, and the term "matter" usually is used in conjunction with a specifying modifier.')